In [2]:
from spatial_split import *

In [3]:
# Generate random dataset with 3 dimensions

from sklearn.datasets import make_blobs
n_samples = 100  # Number of samples
n_features = 3  # Number of dimensions
centers = 3  # Number of clusters
random_state = 42  # Random state for reproducibility

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=centers, random_state=random_state)

In [9]:
split_X, bin_bounds = spatial_split(X, [2,2,2])


In [13]:
from pyspark import SparkContext
from pyspark import SparkConf
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

partition_data = sc.parallelize(split_X).groupByKey().map(lambda x: [x[0], list(x[1])])

In [15]:
partition_data.keys().collect()

[(1, 0, 0), (0, 1, 1), (1, 1, 1)]

In [19]:
p1 = partition_data.filter(lambda x: x[0] == (1, 0, 0))
p1_ind = p1.values().collect()[0]

In [24]:
# calculate grid number for each dimension in one partition
def cal_grid_num(data, eps):
    grid_side_len = eps/np.sqrt(data.shape[1])
    gridnum_each_dim = [int((np.max(data[:, i]) - np.min(data[:, i])) / grid_side_len) for i in range(data.shape[1])]
    return gridnum_each_dim

gridnum_each_dim = cal_grid_num(X[p1_ind], 1)
gridnum_each_dim

[6, 7, 5]

In [ ]:
def get_partition_data(dataset, point_id):
    return dataset[point_id]

def get_grid_cell(partition_data, eps):
    gridnum_each_dim = cal_grid_num(partition_data, eps)
    grid_with_point, _ = spatial_split(partition_data, gridnum_each_dim)
    return grid_with_point

def find_core_point(partition_data, eps, min_pts):
    grid_with_point = get_grid_cell(partition_data, eps)
    core_point = []
    for grid in grid_with_point:
        if len(grid_with_point[grid]) >= min_pts:
            core_point.extend(grid_with_point[grid])
    return core_point

def find_core_cell(partition_data, eps, min_pts):
    core_point = find_core_point(partition_data, eps, min_pts)
    gridnum_each_dim = cal_grid_num(partition_data, eps)
    grid_with_point, _ = spatial_split(partition_data, gridnum_each_dim)
    core_cell = []
    for point in core_point:
        for grid in grid_with_point:
            if point in grid_with_point[grid]:
                core_cell.append(grid)
    return core_cell



In [26]:
grid_data, _ = spatial_split(X[p1_ind], gridnum_each_dim)

In [27]:
grid_data

[[(2, 0, 3), 0],
 [(3, 2, 1), 1],
 [(2, 0, 0), 2],
 [(0, 1, 3), 3],
 [(2, 3, 1), 4],
 [(2, 3, 2), 5],
 [(0, 2, 1), 6],
 [(2, 3, 3), 7],
 [(3, 2, 1), 8],
 [(0, 3, 4), 9],
 [(2, 3, 0), 10],
 [(5, 1, 1), 11],
 [(2, 3, 4), 12],
 [(0, 3, 4), 13],
 [(1, 1, 2), 14],
 [(4, 4, 4), 15],
 [(0, 1, 2), 16],
 [(2, 3, 0), 17],
 [(0, 3, 0), 18],
 [(3, 3, 4), 19],
 [(2, 1, 3), 20],
 [(2, 1, 0), 21],
 [(0, 4, 3), 22],
 [(2, 6, 2), 23],
 [(3, 5, 3), 24],
 [(1, 3, 0), 25],
 [(3, 1, 4), 26],
 [(3, 0, 0), 27],
 [(0, 4, 4), 28],
 [(1, 3, 1), 29],
 [(0, 4, 4), 30],
 [(0, 2, 0), 31],
 [(3, 4, 1), 32]]